# Classify lines of poetry
We will classify lines of poetry as to their literary style: sound poetry or more natural parlando style

Lines are variable length, so we will use an LSTM to encode the line into a fixed-length representation and then add a classificatin layer on top of this fixed-length representation.

The goal of the model is no more to re-create the line (i.e., to remember it as best as possible) but to support the classification. We hence won't train it to recreate the line and won't generate from it anymore. (But this could still be useful as a secondary task in multi-task learning.)

In [ ]:
# import statements for modules that we may need below.
import dynet as dy
import sys
from random import shuffle

# read in the words and set up the "input vocabulary" (in this case: all characters)
data = []
classes = []
with open('parlando.lines') as f:
    data.extend([(0, l.strip()) for l in f.readlines()])
    classes.append('parlando')
with open('soundpoetry.lines') as f:
    data.extend([(1, l.strip()) for l in f.readlines()])
    classes.append('soundpoetry')

characters = set("".join(list([x[1] for x in data])))
characters.add("<EOS>") # special tag that we use to signal end of sequence

int2char = list(characters)
char2int = {c:i for i,c in enumerate(characters)}

VOCAB_SIZE = len(characters)
CLASSES_SIZE = len(classes)

In [ ]:
def compute(rnn, params, instance):
    dy.renew_cg()
    (cls, line) = instance
    lookup = params["lookup"]
    line = ["<EOS>"] + list(line) + ["<EOS>"]
    line = [char2int[c] for c in line]
    s = rnn.initial_state()
    for c in line:
        s = s.add_input(lookup[c])
    R = dy.parameter(params["R"])
    bias = dy.parameter(params["bias"])
    output = R * s.output() + bias
    loss = dy.pickneglogsoftmax(output, cls)
    estimatedClass = max([(v,i) for (i,v) in enumerate(output.value())])[1]
    isCorrect = estimatedClass == cls
    return loss, isCorrect

# train, and report correctness after each training iteration
def train(rnn, params, data):
    shuffle(data)
    trainer = trainer_type(pc)
    for i in range(ITERATIONS):
        correct = 0
        for instance in data:
            loss, isCorrect = compute(rnn, params, instance)
            correct += 1 if isCorrect else 0
            loss_value = loss.value()
            loss.backward()
            trainer.update()
        print("IT: {}, correct: {}".format(i, correct/len(data)))

In [ ]:
ITERATIONS = 20

INPUT_DIM = 20
HIDDEN_DIM = 50
LAYERS = 1

#builder_type = dy.SimpleRNNBuilder
builder_type = dy.LSTMBuilder
#builder_type = dy.GRUBuilder

pc = dy.ParameterCollection()
rnn = builder_type(LAYERS, INPUT_DIM, HIDDEN_DIM, pc)
# add parameters for the hidden->output part for both lstm and srnn
params = {}
params["lookup"] = pc.add_lookup_parameters((VOCAB_SIZE, INPUT_DIM))
params["R"] = pc.add_parameters((CLASSES_SIZE, HIDDEN_DIM))
params["bias"] = pc.add_parameters((CLASSES_SIZE))

trainer_type = dy.SimpleSGDTrainer

In [ ]:
train(rnn, params, data)

This model is still very basic (and we have not evaluated fairly...).
Extensions would be:

- two RNNs for bi-directional analysis (forwards and backwards)
- learning auto-attention for the model to pick up what to pay attention to
- using pre-trained lookup-embeddings and/or pre-train other parameters
